### Konvertierung von GFF-Dateien in das GFF3-Format

Um GFF-Dateien im ursprünglichen Format in das standardisierte GFF3-Format zu überführen, verwenden wir zwei Funktionen: `convert_gff_to_gff3` und `batch_convert_gff_to_gff3`.

Die Funktion `convert_gff_to_gff3` übernimmt eine einzelne `.gff`-Datei und wandelt sie in eine gültige `.gff3`-Datei um. Dabei wird die Attributstruktur in der neunten Spalte angepasst, sodass Schlüssel-Wert-Paare im korrekten GFF3-Format (`key=value`) vorliegen. Zusätzlich wird die obligatorische Kopfzeile `##gff-version 3` eingefügt.

Um diesen Prozess automatisiert auf mehrere Dateien gleichzeitig anzuwenden, nutzen wir die Funktion `batch_convert_gff_to_gff3`. Sie durchläuft ein angegebenes Verzeichnis, sucht nach allen `.gff`-Dateien und ruft für jede Datei `convert_gff_to_gff3` auf. Die konvertierten Dateien werden mit der Endung `.gff3` im Zielverzeichnis gespeichert.

In [2]:
import os
import pandas as pd

# Convert a single .gff file to GFF3 format
def convert_gff_to_gff3(input_path, output_path):
    with open(input_path, 'r') as infile, open(output_path, 'w') as outfile:
        outfile.write("##gff-version 3\n")
        for line in infile:
            line = line.strip()
            if not line or line.startswith("#"):
                continue
            parts = line.split('\t')
            if len(parts) != 9:
                continue
            attributes = parts[8].replace('"', '').replace(' ', '=').replace(';', ';')
            parts[8] = attributes
            outfile.write('\t'.join(parts) + '\n')

# Batch convert all .gff files in a folder to .gff3
def batch_convert_gff_to_gff3(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    for filename in os.listdir(input_dir):
        if filename.endswith(".gff"):
            input_file = os.path.join(input_dir, filename)
            output_file = os.path.join(output_dir, filename.replace(".gff", ".gff3"))
            convert_gff_to_gff3(input_file, output_file)
            print(f"Converted: {filename} -> {os.path.basename(output_file)}")

### Extraktion von Gen-IDs aus GFF3-Dateien

Die Funktion `extract_gene_ids_from_gff3` dient dazu, relevante Gen-IDs aus einer GFF3-Datei zu extrahieren und gleichzeitig die gesamte Datei als DataFrame für weitere Analysen bereitzustellen.
Zunächst wird die GFF3-Datei eingelesen, wobei Kommentarzeilen (beginnend mit `#`) ignoriert werden. Anschließend wird jede gültige Zeile, die genau neun Spalten enthält, in ein `pandas` DataFrame umgewandelt.

Im nächsten Schritt filtert die Funktion gezielt nur jene Einträge heraus, deren Typfeld den Wert `"gene"` enthält. Aus dem Attribut-Feld der GFF3-Datei werden dann zwei mögliche Bezeichner extrahiert:
- `ID=`: die interne eindeutige Kennung eines Gens
- `Name=`: ein möglicherweise alternativer oder gebräuchlicher Genname
Diese beiden Identifier werden kombiniert, um eine vollständige Menge (`set`) aller Gen-IDs zu erstellen. Zusätzlich gibt die Funktion die Anzahl der Geneinträge sowie das gesamte DataFrame zurück.

In [5]:
# Extract gene IDs from GFF3 file and return full DataFrame
def extract_gene_ids_from_gff3(gff3_file):
    with open(gff3_file, 'r') as f:
        lines = [line for line in f if not line.startswith('#')]
    records = [line.strip().split('\t') for line in lines if len(line.strip().split('\t')) == 9]
    df = pd.DataFrame(records, columns=[
        'seqid', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase', 'attributes'
    ])
    gene_df = df[df['type'] == 'gene'].copy()
    gene_df['id_field'] = gene_df['attributes'].str.extract(r'ID=([^;]+)')
    gene_df['name_field'] = gene_df['attributes'].str.extract(r'Name=([^;]+)')
    combined_ids = pd.concat([gene_df['id_field'].dropna(), gene_df['name_field'].dropna()]).unique()
    return set(combined_ids), len(gene_df), df

### Abgleich von Gen-IDs aus Zähltabellen mit GFF3-Dateien

Die Funktion `compare_with_multiple_gff3_and_print_filtered` vergleicht eine gegebene Zähltabelle (Count Table) im `.tsv`-Format mit mehreren GFF3-Dateien in einem angegebenen Verzeichnis. Ziel ist es, festzustellen, welche Gene aus der Zähltabelle in den GFF3-Dateien enthalten sind.

Zunächst wird die Zähltabelle eingelesen und auf jene Zeilen gefiltert, bei denen das Feld `Entity` entweder leer ist oder den Wert `host` enthält – typischerweise handelt es sich dabei um Wirtsgene. Anschließend werden die Gen-IDs aus der ersten Spalte extrahiert.

Die Funktion durchläuft anschließend alle `.gff3`-Dateien im angegebenen Verzeichnis. Für jede Datei wird:

- die Funktion `extract_gene_ids_from_gff3` aufgerufen, um die Gen-IDs zu extrahieren,
- ein Mengenvergleich durchgeführt, um Übereinstimmungen (`matched`) und fehlende Einträge (`unmatched`) zu identifizieren,
- die Übereinstimmungsrate berechnet und ausgegeben.

Wenn es mindestens eine Übereinstimmung gibt, werden die Ergebnisse auf der Konsole ausgegeben. Bei einer Übereinstimmungsrate von unter 100 % wird zusätzlich geprüft, ob sich die fehlenden Gen-IDs zumindest in anderen Feature-Typen oder Attributfeldern der GFF3-Dateien befinden. Diese werden ebenfalls aufgelistet und gezählt.


In [8]:
# Compare count table with each GFF3 file and report matches
def compare_with_multiple_gff3_and_print_filtered(count_table_path, gff3_folder_path):
    counts = pd.read_csv(count_table_path, sep='\t')
    filtered = counts[(counts['Entity'].fillna('') == '') | (counts['Entity'] == 'host')]
    filtered_gene_ids = set(filtered.iloc[:, 0])

    print(f"\nTotal 'host'/empty genes in count table: {len(filtered_gene_ids)}\n")

    for file in os.listdir(gff3_folder_path):
        if file.endswith('.gff3'):
            gff3_path = os.path.join(gff3_folder_path, file)
            gff_gene_ids, gff_total_genes, gff_df = extract_gene_ids_from_gff3(gff3_path)

            matched = filtered_gene_ids & gff_gene_ids
            unmatched = filtered_gene_ids - gff_gene_ids
            match_rate = len(matched) / len(filtered_gene_ids) * 100 if filtered_gene_ids else 0

            if matched:
                print(f"File: {file}")
                print(f"   Genes in GFF3:               {gff_total_genes}")
                print(f"   Matched genes:               {len(matched)}")
                print(f"   Unmatched genes:             {len(unmatched)}")
                print(f"   Match rate:                  {match_rate:.2f}%")

                if match_rate < 100.0:
                    search_results = []
                    for gene_id in unmatched:
                        found = gff_df['attributes'].str.contains(gene_id, na=False)
                        matches = gff_df[found]
                        search_results.append({
                            'Unmatched Gene ID': gene_id,
                            'Found in any attribute': not matches.empty,
                            'Example attribute match': matches.iloc[0]['attributes'] if not matches.empty else ""
                        })
                    found_count = sum(1 for r in search_results if r['Found in any attribute'])
                    print(f"   Found in other feature types: {found_count}")
                    print(f"   First {len(unmatched)} unmatched gene IDs: {list(unmatched)[:10]}\n")

### Automatisierte Verarbeitung von Count-Tabellen und GFF-Dateien

Die Funktion `run_all` führt zwei Schritte automatisiert aus:

1. **Konvertierung**: Alle `.gff`-Dateien im Eingabeverzeichnis werden ins GFF3-Format umgewandelt.
2. **Vergleich**: Alle `.tsv`-Count-Tabellen werden mit den konvertierten `.gff3`-Dateien abgeglichen, um Gen-Übereinstimmungen zu prüfen.

So lassen sich mehrere Datensätze effizient in einem Durchlauf analysieren.


In [10]:
# Apply analysis to all count tables in a folder
def run_all(counts_folder_path, gff_folder_path, gff3_folder_path):
    print("\nStep 1: Converting .gff to .gff3")
    batch_convert_gff_to_gff3(gff_folder_path, gff3_folder_path)

    print("\nStep 2: Comparing count tables to GFF3 files")
    for file in os.listdir(counts_folder_path):
        if file.endswith(".tsv"):
            count_file_path = os.path.join(counts_folder_path, file)
            print(f"\nProcessing count table: {file}")
            compare_with_multiple_gff3_and_print_filtered(count_file_path, gff3_folder_path)

Bei den Ergebnissen fällt auf, dass einige Datensätze keine vollständige Übereinstimmung (100 %) mit den jeweiligen GFF3-Dateien aufweisen. Die betroffenen Gene sollten genauer betrachtet werden.

In [12]:
run_all("counts/", "hostgff/", "hostgff3/")


Step 1: Converting .gff to .gff3
Converted: Enterobacteria_host.genomic.gff -> Enterobacteria_host.genomic.gff3
Converted: new_Clostridioides_difficile_host_genomic.gff -> new_Clostridioides_difficile_host_genomic.gff3
Converted: new_vibrio_host_genomic.gff -> new_vibrio_host_genomic.gff3
Converted: new_Yersinia_host_genome.gff -> new_Yersinia_host_genome.gff3
Converted: pseudomonas_host_genomic.gff -> pseudomonas_host_genomic.gff3
Converted: Vibrio_genomic.gff -> Vibrio_genomic.gff3

Step 2: Comparing count tables to GFF3 files

Processing count table: Brandao_LB_full_raw_counts.tsv

Total 'host'/empty genes in count table: 5665

File: pseudomonas_host_genomic.gff3
   Genes in GFF3:               5678
   Matched genes:               5665
   Unmatched genes:             0
   Match rate:                  100.00%

Processing count table: Ceyssens_non-directional_full_raw_counts.tsv

Total 'host'/empty genes in count table: 5665

File: pseudomonas_host_genomic.gff3
   Genes in GFF3:     